## Quantization Enforcement and Assertion
We found out that qkeras does not enforce the quantization of the layers during training: it simply learns the alpha values so the activations can be later scaled down according to

$y = \alpha x + b$

This forces us to make sure that we apply our quantization scheme on each layer. Thus we had to create classes for specifically enforcing the kernel/bias/activation quantization/initialization/constraint within the quantization range and limits. 

In this notebook we simply iterate over all pre-defined models, build them (with default shapes), and we assert whether all the weights and network parameters are within range. 

Then we pass some random data (normalized) and check that all the activations are also within range

In [ ]:
import sys 
sys.path.append('..')

# Import datetime to get today's date
from datetime import datetime

""" Let's add our custom netsurf code """
import netsurf

""" Get netsurf path """
import os 
parent_dir = os.path.dirname(os.path.dirname(os.path.abspath(netsurf.__file__)))

In [ ]:
""" Initialize a pergamos document for our output report """
import pergamos as pg

# Set variables
qscheme = "q<6,2,1>"
pruning = 0.0
prerank = True
""" First of all, let's define a quantization Scheme """
Q = netsurf.QuantizationScheme(qscheme)
print(Q)
#benchmarks = ['mnist_fnn', 'autompg', 'smartpixel_small']
benchmarks = ['fashion_mnist_fnn']

# Set filename
benchmarks_dir = os.path.join(parent_dir, 'benchmarks')
datasets_dir = os.path.join(parent_dir, 'datasets')

filename = f"2_qpolar_{Q._scheme_str.no_special_chars()}_bmarks_{'_'.join(benchmarks)}_prune{str(pruning).replace('.','_')}.html"
print(filename)
doc = pg.Document(filename, theme="default")
doc.required_scripts.add('mathjax')


""" Add a title to the document """
doc.append(pg.Markdown(f"""# Benchmarks Quantization Assertion
> Author: Manuel B Valentin

> Creation date: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}

> Project: netsurf

> Used packages: netsurf, tensorflow, numpy, matplotlib, pergamos
        
"""))

# Add a tab container
tabs = pg.TabbedContainer({'Motivation': [], 
                           'Pre-training analysis': [],
                           'Training': [],
                            'Post-Training': [],
                           'BER Injection': [],
                           'Conclusions': []})

# Get individual tabs
tabmotivation = tabs['Motivation']
tabpretraining = tabs['Pre-training analysis']
tabtraining = tabs['Training']
tabposttraining = tabs['Post-Training']
tabber = tabs['BER Injection']
tabconclusions = tabs['Conclusions']

# Add to documnt
doc.append(tabs)


In [ ]:
# Add a markdown description of what we want to achieve with this report in the first tab
md = r"""
## 1. Loss Taylor expansion

Given a loss function $\mathcal{L}(w)$, where $w$ is the vector of all weights in the network, the Taylor expansion around some point $w_0$ (say, the trained weights) for a small perturbation $\Delta w$ is:

$$\mathcal{L}(w_0 + \Delta w) \approx \mathcal{L}(w_0) + \nabla \mathcal{L}(w_0)^T \Delta w + \frac{1}{2} \Delta w^T H \Delta w$$

Where:

* $\nabla \mathcal{L}(w_0)$ is the gradient vector of the loss at w_0
* $H$ is the Hessian matrix, i.e. $H = \nabla^2 \mathcal{L}(w_0)$

---

## 2. If the model is trained…

If the model has been well trained, then:

$\nabla \mathcal{L}(w_0) \approx 0$

Because you're sitting near a (local) minimum.

This removes the linear term:
$\mathcal{L}(w_0 + \Delta w) - \mathcal{L}(w_0) \approx \frac{1}{2} \Delta w^T H \Delta w$

So the change in loss caused by a perturbation $\Delta w$ is approximately:
$\Delta \mathcal{L} \approx \frac{1}{2} \Delta w^T H \Delta w$

---

## 3. Interpretation for bit flips

A bit flip in the quantized weights causes a small but structured change in the weights:

 * Say, flipping the 3rd bit in weight $w_i$ causes it to change by $\delta_i$, so:

$\Delta w = \begin{bmatrix}
0 \\ \cdots \\ \delta_i \\ \cdots \\ 0
\end{bmatrix}$

Then the loss increase is (approximately):
$\Delta \mathcal{L} \approx \frac{1}{2} \delta_i^2 H_{ii}$

If multiple bits are flipped across weights, you sum their pairwise interactions via H, including off-diagonal terms (if not ignored).

--- 

## 4. Implications for ranking

This approximation motivates ranking bit positions (or weights) by:

* $\delta^2 \cdot H_{ii}$: bit-flip magnitude times curvature
* This is the FKeras method: estimates $H_{ii}$ and ranks accordingly
* You could generalize it to your method:
 $\text{Impact} \cdot H$, not just gradients

---

## 5. When does this approximation hold?

✅ Works well when:

* Bit-flip magnitude is small (i.e., local region)
* Model is near a minimum
* Hessian is stable (not exploding)

❌ Fails when:

* Model isn’t trained well (gradient is large)
* Loss surface is highly non-quadratic

---

## Summary

The formula:
$\Delta \mathcal{L} \approx \frac{1}{2} \Delta \mathbf{w}^T H \Delta \mathbf{w}$

tells us how bit-flips propagate into loss increases, and explains why the Hessian is so powerful for ranking robustness. 
It encodes:

* How impactful a perturbation is (via $\delta$)
* How sensitive the loss is locally (via $H$)

"""

# Create markdown
md = pg.Markdown(md)

# Add to the first tab
tabmotivation.append(md)

""" Add quantization container to doc report """
tabpretraining.extend(Q.html())

""" Save doc to file (we save after adding each element) """
doc.save(filename)

In [ ]:
# Let's create a container for all benchmarks 
benchmark_ct = pg.CollapsibleContainer("🧺 Benchmarks", layout='vertical')

# And another one for tabtraining
benchmark_sessions_ct = pg.CollapsibleContainer("🧺 Benchmarks", layout='vertical')

# And another one for tabposttraining
benchmark_posttraining_ct = pg.CollapsibleContainer("🧺 Benchmarks", layout='vertical')

# And yet another for "BER Injection"
benchmark_ber_ct = pg.CollapsibleContainer("🧪 Experiments", layout='vertical')

""" Add to documnt """
tabpretraining.append(benchmark_ct)

""" Add """
tabtraining.append(benchmark_sessions_ct)

""" Add """
tabposttraining.append(benchmark_posttraining_ct)

""" Add """
tabber.append(benchmark_ber_ct)

# Define benchmarks to analyze
#benchmarks = ['dummy', 'mnist_hls4ml', 'autompg', 'smartpixel_small', 'smartpixel_large',
#              'cifar10', 'mnist_lenet5', 'ECONT_AE'
# 'cifar100', 'svhn', 'fashion_mnist', 'imdb', 'reuters', 'boston_housing']
# TODO: Fix visualization/contrast for cifar10
# TODO: mnist_lenet5 seems to be working (good accuracy), but I'm not too happy about the alphas/betas. Some layers still have a big portion outside of the valid interval

config_per_methods = netsurf.config.config_per_method
protection_range = netsurf.config.DEFAULT_PROTECTION
ber_range = netsurf.config.DEFAULT_BER

#methods = ['qpolar', 'qpolargrad', 'bitwise_msb', 'random', 'hirescam_norm', 
#           'hiresdelta', 'hessian', 'hessiandelta', 'weight_abs_value']
methods = ['hessian', 'fisher', 'qpolar', 'qpolargrad', 'bitwise_msb', 'random', 'hirescam_norm', 'hiresdelta', 'weight_abs_value']

# Loop for benchmarks
for benchmark_name in benchmarks:
    # Create benchmark object 
    bmk = netsurf.get_benchmark(benchmark_name, Q,  benchmarks_dir = benchmarks_dir,
                                datasets_dir = datasets_dir, pruning = pruning,
                                load_weights = False)
    
    # Add benchmark html to container (this includes model + dataset htmls)
    # (run before training the model...)
    benchmark_ct.append(bmk.html())

    # Now let's prepare the data
    nsample_mod = 48 if 'ECON' in bmk.name else -1
    XYTrain = netsurf.utils.prepare_data(bmk, subset = 'train', nsample_mod = nsample_mod)

    # Initialize the uncertainty profiler (pre-training)
    pre_robustness_sgn_path = os.path.join(bmk.model_dir, 'uncertainty_signatures', f'{benchmark_name}.pretraining.netsurf.sgn')
    pre_robustness_sgn = netsurf.UncertaintyProfiler.profile(bmk.model, XYTrain, bmk.model.loss, 
                                                             batch_size = 2000, filepath = pre_robustness_sgn_path,
                                                             verbose = True)
    # Save 
    pre_robustness_sgn.save_to_file(pre_robustness_sgn_path)
    
    # Add profile to tabpretraining
    benchmark_ct.append(pre_robustness_sgn.html())

    # Now we can reload the weights (After the profiling)
    bmk.load_weights(verbose = True)

    # TRAINING - SESSION
    # Try to get a session (if not, train)
    sess = netsurf.get_training_session(bmk, show_plots = False, plot = True)

    # Create a container for this bmk in tabtraining
    bmk_sess_ct = pg.CollapsibleContainer(benchmark_name, layout='vertical')

    # Add session to tabtraining
    bmk_sess_ct.append(sess.html())

    # And add to benchmarks in tabtraining
    benchmark_sessions_ct.append(bmk_sess_ct)

    # Add benchmark again to post-training to check how the weights changed
    benchmark_posttraining_ct.append(bmk.html())

    # Initialize the uncertainty profiler (post-training)
    post_robustness_sgn_path = os.path.join(bmk.model_dir, 'uncertainty_signatures', f'{benchmark_name}.posttraining.netsurf.sgn')
    post_robustness_sgn = netsurf.UncertaintyProfiler.profile(bmk.model, XYTrain, bmk.model.loss, 
                                    batch_size = 2000, verbose = True, filepath = post_robustness_sgn_path)

    # Save 
    post_robustness_sgn.save_to_file(post_robustness_sgn_path)
    
    # Add profile to tabposttraining
    benchmark_posttraining_ct.append(post_robustness_sgn.html())

    # Let's compare the divergence between the profiles
    div_profile = netsurf.ProfileDivergence.from_signatures(pre_robustness_sgn, post_robustness_sgn)

    #div_profile.plot_divergence_summary()
    #div_profile.plot_advanced_divergence_summary()

    """ Save doc to file (we save after adding each element) """
    doc.save(filename)
    
    # Create a container for this benchmark in "BER Injection"
    bmk_ber_ct = pg.CollapsibleContainer(benchmark_name, layout='vertical')

    # Add to tabber
    benchmark_ber_ct.append(bmk_ber_ct)

    # Create our ranker profiler 
    rankers_bucket = netsurf.RankingComparator.from_directory(bmk.experiments_dir, rankers = methods, quantization = Q, 
                                                              config_per_methods = config_per_methods)

    # Loop thru methods
    exps = {}
    for method in methods:
        #################################################################
        # 1. Perform ranking according to method
        #################################################################
        # Rank weights 
        ranking = rankers_bucket.rank(method, bmk.model, *XYTrain, verbose = True)
        # Save rank to csv file 
        ranking.save(overwrite = False)

        #################################################################
        # 2. Create experiment object
        #################################################################
        exp = netsurf.Experiment(bmk, ranking, num_reps = -1, 
                                 ber_range = ber_range, 
                                 protection_range = protection_range, 
                                 verbose = True)
    
        # Print experiment info 
        print(exp)

        #################################################################
        # 3. Run experiment with given ranking and for whatever 
        #       range of protection and rad 
        #################################################################
        #batch_size = 1000,
        exp.run_experiment(bmk, XYTrain,
                           batch_size = None,
                           ber_range = ber_range, 
                           protection_range = protection_range, 
                           rerun = False)
        
        # Add experiment to container
        bmk_ber_ct.append(exp.html())

        # Save experiment object
        exp.save()

        # Add to dict
        exps[method] = exp

        """ Save doc to file (we save after adding each element) """
        doc.save(filename)

        raise ValueError("Stop here for now")

In [ ]:
config_per_methods['hessian']

In [ ]:
import pandas as pd
from netsurf.gui.plotter import *

import pandas as pd 
import matplotlib.pyplot as plt 

def plot_barplot(subplotters, 
                 ax = None, y = 'mean', metric = None,  
                 ylims = None, title = None, xlog = False, ylog = False,
                 show = False, info_label = None, standalone = True, 
                 baseline = None, remove_baseline = False, single_out = 'random', 
                 cmap = 'viridis', filename = None, ylabel = None,
                 **kwargs):

    # Loop thru each plotter and get the VUSs
    VUCs = []
    for method in subplotters:
        # Loop thru configs 
        for config in subplotters[method]:
            # Get the plotter obj
            plotter = subplotters[method][config]
            # Get the vuc
            VUCs += [{'method': method, 'config': config, 'vus': plotter.vus.loc['vus'][y]}]

    # Convert VUCs to a dataframe
    df = pd.DataFrame(VUCs)
    # Sort by vus 
    df = df.sort_values('vus', ascending = metric.lower() not in ['accuracy', 'acc'])

    # if remove_baseline
    if remove_baseline:
        if baseline in df['method'].values:
            # subtract the baseline from the vus
            df['vus'] = df['vus'] - df[df['method'] == baseline]['vus'].values[0]

    # Xrange is always the number of methods
    # ylims
    if ylims is None:
        ylims = (0, 1.1*df['vus'].max())

    # Create color mapper 
    # Get the min and max values
    vmin = df['vus'].min()
    vmax = df['vus'].max()
    # Create a color palette
    cmap = plt.get_cmap(cmap)
    # Normalize the values
    norm = plt.Normalize(vmin, vmax)
    # Create lambda function to map any value to color later 
    color_mapper = lambda x: cmap(norm(x))
    hatch_styles = {'auc': '**', 'vus': '//'}

    # If ax is none, create a new figure
    if ax is None or standalone:
        fig, ax = plt.subplots()
        wsbmr.utils.mark_figure_as_deletable(fig)
    else:
        fig = ax.figure
    
    # Initialize the width, x of the bar
    bar_w = 0.4
    bar_space = 0.1 # Space between method bars (same method are not spaced)

    """ Pyplot configuration for hatches """
    # Store old value of 'hatc.linewidth'
    old_linewidth = plt.rcParams['hatch.linewidth']
    # Set the linewidth of the hatch lines
    plt.rcParams['hatch.linewidth'] = 0.3
    plt.rcParams["lines.solid_capstyle"] = "butt"
    old_grid_color = plt.rcParams['grid.color']
    plt.rcParams['grid.color'] = (0.5, 0.5, 0.5, 0.3)

    # Group by method 
    g = df.groupby('method', sort = False)

    # Set ylims 
    ax.set_ylim(ylims)
    # Set ticks params here (we need them to get the size of the xticklabels)
    ax.tick_params(axis='x', labelsize=9) 

    xticks = []
    xticklabels = []

    # Find the position of the corresponding xtick label
    label = ax.get_xticklabels()[0]  # Get the Text object for the label

    # Use `get_window_extent` to find the label's extent in display space (optional)
    renderer = fig.canvas.get_renderer()
    bbox = label.get_window_extent(renderer=renderer)

    # Convert the bounding box to data coordinates
    inv = ax.transData.inverted()
    bbox_data = inv.transform(bbox)

    # Use the bottom of the bbox as the y-coordinate for the line
    label_y_offset = bbox_data[1][1]  # The top edge of the label in data coordinates
    label_new_line_height = bbox_data[1][1] - bbox_data[0][1]  # The height of the label in data coordinates

    # Now loop thru methods
    bars = []
    for i, (method, group) in enumerate(g):
        # Loop thru configs, vus
        nconfigs = len(group)
        for j, (config, vus) in enumerate(zip(group['config'], group['vus'])):
            # Get the x position of the bar
            x = i*(bar_w + bar_space) + j*bar_w
            
            # Now get all four coordinates, for simplicity 
            x0, x1, y0, y1 = x, x + bar_w, 0, vus

            # Get this bar's value
            c = color_mapper(y1)
            hs = hatch_styles['vus']
            if single_out is not None:
                if single_out == method:
                    #c = (1, 1, 1, 1) if btype == 'AUC' else (0, 0, 0, 1)
                    c = (0, 0, 0, 1)
                    hs = 'o'

            # Add rectangle (we will delete the old bar)
            bar = ax.fill_between([x0, x1], y0, y1, color=c[:-1] + (0.3,), edgecolor = 'k', linewidth = 0.5, label = method)
            # Add hatch to this patch we just created 
            bar.set_hatch(hs)
            bars.append(bar)

            # Add a label on top of the bar with the value of the VUS
            ax.text(x + bar_w/2, vus + label_new_line_height/2, f'{vus:.3f}', ha='center', va='bottom', fontsize=9)

            # Add xlabel to list 
            xticks += [x + bar_w/2]
            sp = "".join(['\n']*((i+j)%2))
            mstr = method.replace('_', '\n').replace(' ', '\n')
            mstr = mstr.replace('delta', r'$\Delta$')
            xticklabels += [f'{sp}{mstr}\n{config}' if nconfigs > 1 else f'{sp}{mstr}']

            # Add a line to connect the bar to the xticklabel underneath (only if i+j is odd)
            if (i+j) % 2 == 1:
                line = mlines.Line2D(
                    [x + bar_w/2, x + bar_w/2],           # x-coordinates
                    [0, label_y_offset - label_new_line_height*0.7],        # y-coordinates (outside the plot area)
                    color="black",
                    lw=0.8
                )
                line.set_clip_on(False)  # Ensure the line is not clipped by the axis
                ax.add_artist(line)     # Add the line as an artist

    # Set grid to dashed and also turn minor grid on
    ax.grid(which='major', linestyle='--')
    ax.minorticks_on()
    ax.grid(which='minor', linestyle=':')

    # Set xticks and xticklabels
    ax.set_xticks(xticks)
    ax.set_xticklabels(xticklabels, rotation = 0, ha = 'center')

    # Setup labels correctly
    # parse ylabel
    if ylabel:
        ylabel = ylabel.replace('mae', 'Mean Absolute Error').replace('mse', 'Mean Squared Error').replace('accuracy', 'Accuracy')
        ax.set_ylabel(ylabel)

    # Set scale
    if xlog: ax.set_xscale('log')
    if ylog: ax.set_yscale('log')

    # Setup the title 
    if title: ax.set_title(title)

    # Add the info label
    t = None
    if len(info_label) > 0:
        # Get axis position in figure-relative coordinates
        axis_position = ax.get_position()  # Returns (x0, y0, width, height)
        y_top = axis_position.y1 + 0.03  # Slightly above the top of the axis (in figure-relative coordinates)
        # Create label
        t = create_label(ax, info_label, 0.5, y_top, fontsize=9, border_color="black", padding=0.5, num_columns=2)
    
    if show:
        plt.show(block=False)  # Show without blocking the PyQt5 event loop
    else:
        if filename is not None:
            plt.savefig(filename, bbox_inches='tight')
            plt.close()

    """ Restore the default configuration """
    # Set the 'hatch.linewidth' back to its original value
    plt.rcParams['hatch.linewidth'] = old_linewidth
    # Set the 'grid.color' back to its original value
    plt.rcParams['grid.color'] = old_grid_color

    return ax.figure, ax, t, bars

def plot_boxplot(subplotters, 
                 ax = None, y = 'mean', metric = None, colors = None,
                 ylims = None, title = None, xlog = False, ylog = False,
                 show = False, info_label = None, standalone = True, 
                 baseline = None, remove_baseline = False, single_out = 'random', 
                 cmap = 'seismic', filename = None, ylabel = None,
                 sorter = 'mean',
                 **kwargs):

    # Assert sorter 
    assert sorter in ['median', 'mean', 'max', 'min', 'std'], 'Invalid sorter'

    # Loop thru each plotter and get the VUSs
    VUCs = []
    for method in subplotters:
        # Loop thru configs 
        for config in subplotters[method]:
            # Get the plotter obj
            plotter = subplotters[method][config]
            # Get the vuc
            VUCs += [{'method': method, 'config': config, **plotter.vus.loc['vus'].to_dict()}]

    # Convert VUCs to a dataframe
    df = pd.DataFrame(VUCs)
    # Sort by vus 
    df = df.sort_values(sorter, ascending = metric.lower() not in ['accuracy', 'acc'])

    # Xrange is always the number of methods
    # ylims
    if ylims is None:
        ylims = (0.95*df['min'].min(), 1.05*df['max'].max())

    # Create color mapper 
    # Get the min and max values
    vmin = (df['median'] - df['std']).min()
    vmax = (df['median'] + df['std']).max()
    # Create a color palette
    cmap = plt.get_cmap(cmap)
    # Normalize the values
    norm = plt.Normalize(vmin, vmax)
    # Create lambda function to map any value to color later 
    color_mapper = lambda x: cmap(norm(x))
    hatch_styles = {'auc': '**', 'vus': '//'}

    # If ax is none, create a new figure
    if ax is None or standalone:
        fig, ax = plt.subplots(figsize = (7, 10))
        netsurf.utils.mark_figure_as_deletable(fig)
    else:
        fig = ax.figure
    
    # Initialize the width, x of the bar
    bar_w = 0.2
    bar_space = 0.1 # Space between method bars (same method are not spaced)
    lw = 0.0125

    # Store old value of 'hatc.linewidth'
    old_linewidth = plt.rcParams['hatch.linewidth']
    # Set the linewidth of the hatch lines
    plt.rcParams['hatch.linewidth'] = 0.3
    plt.rcParams["lines.solid_capstyle"] = "butt"
    old_grid_color = plt.rcParams['grid.color']
    plt.rcParams['grid.color'] = (0.5, 0.5, 0.5, 0.3)

    # Group by method 
    g = df.groupby('method', sort = False)

    # Set ylims 
    ax.set_ylim(ylims)
    # Set ticks params here (we need them to get the size of the xticklabels)
    ax.tick_params(axis='x', labelsize=9) 

    xticks = []
    xticklabels = []

    # Find the position of the corresponding xtick label
    label = ax.get_xticklabels()[0]  # Get the Text object for the label

    # Use `get_window_extent` to find the label's extent in display space (optional)
    renderer = fig.canvas.get_renderer()
    bbox = label.get_window_extent(renderer=renderer)

    # Convert the bounding box to data coordinates
    inv = ax.transData.inverted()
    bbox_data = inv.transform(bbox)

    # Use the bottom of the bbox as the y-coordinate for the line
    label_y_offset = bbox_data[1][1]  # The top edge of the label in data coordinates
    label_new_line_height = bbox_data[1][1] - bbox_data[0][1]  # The height of the label in data coordinates

    # Now loop thru methods
    boxes = []
    x = 0
    for i, (method, group) in enumerate(g):
        # Loop thru configs, vus
        nconfigs = len(group)
        for j, (_, row) in enumerate(group.iterrows()):
        
            config = row['config']
            median = row['median']
            std = row['std']
            max = row['max']
            min = row['min']

            # Get the x position of the box
            x = i*(bar_w + bar_space) + j*bar_w
            
            # Now get all four coordinates, for simplicity 
            x0, x1, y0, y1, ym = x, x + bar_w, median - std, median + std, median

            # Get this bar's value
            c = color_mapper(y1)
            hs = hatch_styles['vus']
            if single_out is not None:
                if single_out == method:
                    #c = (1, 1, 1, 1) if btype == 'AUC' else (0, 0, 0, 1)
                    c = (0, 0, 0, 1)
                    hs = 'o'

            # Create a box going from the median to the top of the box
            # The color of this box will be the equivalent of the bottom value of the box 
            ptop = ax.fill_between([x0, x1], ym, y1, 
                                    color=c[:-1] + (0.3,), 
                                    edgecolor = 'k', 
                                    linewidth = 0.5)
            # Add hatch to this patch we just created 
            ptop.set_hatch('//')

            # And now one from the median down to the bottom of the box
            pbot = ax.fill_between([x0, x1], y0, ym, 
                                    color=c[:-1] + (0.7,), 
                                    edgecolor = 'k', 
                                    linewidth = 0.5)
            # Add hatch to this patch we just created
            pbot.set_hatch('\\\\')

            # Add a line for the median
            m = ax.plot([x0, x1], [ym, ym], color='k', linewidth = 1.4)

            """ Add whiskers now """
            # Top whisker 
            wx0, wx1, wy0, wy1 = x + bar_w/2 - lw/2, x + bar_w/2 + lw/2, median + std, max
            # try this with a patch instead of a line 
            warnings.filterwarnings("ignore")
            ax.imshow([[wy1, wy1], [wy0, wy0]], 
                cmap = cmap, 
                extent = [wx0, wx1, wy0, wy1],
                interpolation = 'bicubic', 
                vmin = vmin, vmax = vmax,
                alpha = 0.8
            )
            
            # Create a Rectangle patch with the desired border color
            rect = plt.Rectangle((wx0, wy0), lw, wy1-wy0, 
                                edgecolor='k', linewidth = 0.4, facecolor='none')
            # Add the patch to the Axes
            ax.add_patch(rect)
            # Finally, add the whisker line (horizontal line)
            ax.plot([x + bar_w/2 - lw, x + bar_w/2 + lw], [wy0, wy0], color='k', linewidth = 0.1)

            # Bottom whisker 
            wx0, wx1, wy0, wy1 = x + bar_w/2 - lw/2, x + bar_w/2 + lw/2, min, median - std
            # try this with a patch instead of a line 
            warnings.filterwarnings("ignore")
            ax.imshow([[wy1, wy1], [wy0, wy0]], 
                cmap = cmap, 
                extent = [wx0, wx1, wy0, wy1],
                interpolation = 'bicubic', 
                vmin = vmin, vmax = vmax,
                alpha = 0.8
            )

            # Create a Rectangle patch with the desired border color
            rect = plt.Rectangle((wx0, wy0), lw, wy1-wy0, 
                                edgecolor='k', linewidth = 0.4, facecolor='none')
            # Add the patch to the Axes
            ax.add_patch(rect)
            # Finally, add the whisker line (horizontal line)
            ax.plot([x + bar_w/2 - lw, x + bar_w/2 + lw], [wy1 + 0.01, wy1 - 0.01], color='k', linewidth = 1)
            line = mlines.Line2D(
                    [x + bar_w/2 - lw, x + bar_w/2 + lw],           # x-coordinates
                    [wy1, wy1],        # y-coordinates (outside the plot area)
                    color="black",
                    lw=0.8
                )
            # Add the line to the plot
            ax.add_artist(line)

            # # Now let's get the actual data points for this method 
            # dfm = subplotters[method][config].curves
            # # Make sure len(dfm) > 1, otherwise just pick the only point
            # points = dfm['auc'].values
            # points_tmrs = dfm['tmr'].values
            # if 'tmr_color' in dfm:
            #     pointcols = dfm['tmr_color'].values
            # else:
            #     pointcols = [cmapper(p) for p in points_tmrs]
            
            # # Get the color for each point 
            # #pointcols = [cmapper(p) for p in points]
            # ax.scatter([i+1.5]*len(points), points, edgecolor = 'k', linewidth = 0.4, alpha = 0.8, color = pointcols, s = 30)
            
            # Append to boxes
            #boxes.append([ptop, pbot])

            # Add a label on top of the box with the value of the VUS
            ax.text(x + bar_w/2, max + label_new_line_height/2, f'{median:.3f}', ha='center', va='bottom', fontsize=9)

            # Add xlabel to list 
            xticks += [x + bar_w/2]
            sp = "".join(['\n']*((i+j)%2))
            mstr = method.replace('_', '\n').replace(' ', '\n')
            mstr = mstr.replace('delta', r'$\Delta$')
            xticklabels += [f'{sp}{mstr}\n{config}' if nconfigs > 1 else f'{sp}{mstr}']

            # Add a line to connect the bar to the xticklabel underneath (only if i+j is odd)
            if (i+j) % 2 == 1:
                line = mlines.Line2D(
                    [x + bar_w/2, x + bar_w/2],           # x-coordinates
                    [-label_new_line_height, 2*label_new_line_height],        # y-coordinates (outside the plot area)
                    color="black",
                    lw=0.8
                )
                line.set_clip_on(False)  # Ensure the line is not clipped by the axis
                ax.add_artist(line)     # Add the line as an artist

    # Set grid to dashed and also turn minor grid on
    ax.grid(which='major', linestyle='--')
    ax.minorticks_on()
    ax.grid(which='minor', linestyle=':')

    # Update xlims
    ax.set_xlim(-bar_w/2 - bar_space, x + bar_w + bar_space)

    # Set xticks and xticklabels
    ax.set_xticks(xticks)
    ax.set_xticklabels(xticklabels, rotation = 0, ha = 'center')

    # Setup labels correctly
    # parse ylabel
    if ylabel:
        ylabel = ylabel.replace('mae', 'Mean Absolute Error').replace('mse', 'Mean Squared Error').replace('accuracy', 'Accuracy')
        ax.set_ylabel(ylabel)

    # Set scale
    if xlog: ax.set_xscale('log')
    if ylog: ax.set_yscale('log')

    # Setup the title 
    if title: ax.set_title(title)

    ax.set_aspect('auto')
    # if vmin == vmax:
    #     vmin = 0
    #     vmax = 1
    # if np.isnan(vmin) or np.isnan(vmax):
    #     vmin = 0
    #     vmax = 1
    # if np.isinf(vmin) or np.isinf(vmax):
    #     vmin = 0
    #     vmax = 1
    # ax.set_ylim(0.95*vmin, 1.05*vmax)
    ax.set_ylim(ylims)

    # Add the info label
    t = None
    if len(info_label) > 0:
        # Get axis position in figure-relative coordinates
        axis_position = ax.get_position()  # Returns (x0, y0, width, height)
        y_top = axis_position.y1 + 0.03  # Slightly above the top of the axis (in figure-relative coordinates)
        # Create label
        t = create_label(ax, info_label, 0.5, y_top, fontsize=9, border_color="black", padding=0.5, num_columns=2)
    
    if show:
        plt.show(block=False)  # Show without blocking the PyQt5 event loop
    else:
        if filename is not None:
            plt.savefig(filename, bbox_inches='tight')
            plt.close()

    """ Restore the default configuration """
    # Set the 'hatch.linewidth' back to its original value
    plt.rcParams['hatch.linewidth'] = old_linewidth
    # Set the 'grid.color' back to its original value
    plt.rcParams['grid.color'] = old_grid_color


    return ax.figure, ax, t, boxes


In [ ]:
# Create a container for the comparison plots 
comparison_ct = pg.CollapsibleContainer("🧺 Comparison", layout='vertical')

# Add to documnt
tabber.append(comparison_ct)

for m in bmk.metric_names:
    # Create a container for the comparison plots
    ct = pg.CollapsibleContainer(f'{m}', layout = 'vertical')

    # Add to group
    comparison_ct.append(ct)

    _m = m.lower()

    subplotters = {}
    for method, exp in exps.items():
        if method not in subplotters:
            subplotters[method] = dict()
        subplotters[method][exp.name] = netsurf.gui.plotter.ExperimentsPlotter(exp.results, 
                                                                            metric = _m)



    # Create a container for the barplot
    ct2 = pg.CollapsibleContainer('📊 Bar plot', layout = 'vertical')

    fig, ax = plt.subplots(1,1, figsize = (10, 5))


    plot_barplot(subplotters, 
                    ax = ax, y = 'mean', metric = _m,
                    info_label=[], standalone = False, remove_baseline = True, baseline = 'random')  
                    #  ylims = None, title = None, xlog = False, ylog = False,
                    #  show = False, standalone = True, 
                    #  baseline = None, remove_baseline = False, single_out = 'random', 
                    #  cmap = 'viridis', filename = None, ylabel = None,
                    #  **kwargs)

    # Create plot img 
    p = pg.Plot(fig)
    ct2.append(p)
    # Append to group
    ct.append(ct2)
    # Close ax 
    plt.close(fig)

    # And now the same for boxplot
    # Create a container for the barplot
    ct3 = pg.CollapsibleContainer('⧮ Box plot', layout = 'vertical')

    fig2, ax2 = plt.subplots(1,1, figsize = (10, 5))

    plot_boxplot(subplotters, 
                    ax = ax2, y = 'mean', metric = _m,
                    info_label=[], standalone = False, remove_baseline = True, baseline = 'random')  

    # Create plot img 
    p = pg.Plot(fig2)
    ct3.append(p)
    # Append to group
    ct.append(ct3)
    # Close ax 
    plt.close(fig2)

    # Save doc
    doc.save(filename)
